# 1. Import libary

In [1]:
import os
import time
import pickle
import mediapipe as mp
import cv2
import matplotlib.pyplot as plt
import numpy as np
from PIL import ImageFont, ImageDraw, Image
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import string

# Hàm sữ dụng được tiếng việt

In [2]:
# Tải font chữ hỗ trợ tiếng Việt (ví dụ: Arial Unicode MS hoặc Noto Sans)
font_path = "arial.ttf"  # Bạn cần có file font .ttf hỗ trợ Unicode
font = ImageFont.truetype(font_path, 32)  # Kích thước chữ

def draw_text_vietnamese(image, text, position, text_color=(0, 255, 255)):
    """Hiển thị văn bản tiếng Việt trên ảnh bằng Pillow"""
    # Chuyển ảnh từ OpenCV sang PIL
    pil_img = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    draw = ImageDraw.Draw(pil_img)
    
    # Vẽ chữ lên ảnh
    draw.text(position, text, font=font, fill=text_color)
    
    # Chuyển ảnh từ PIL về OpenCV
    return cv2.cvtColor(np.array(pil_img), cv2.COLOR_RGB2BGR)

# 2. Collect image

In [ ]:
# https://www.youtube.com/watch?v=6_gXiBe9y9A video về ngôn ngữ ký hiệu cho alphabet
# hành động để phát hiện 0-9 a-z
actions = np.array([str(i) for i in range(10)] + [chr(i) for i in range(97, 123)])

In [7]:
# hành đồng từ 0-4
actions = np.array(['0', '1', '2', '3', '4'])

In [ ]:
# hành đồng từ 5-9
actions = np.array(['5', '6', '7', '8', '9'])

In [ ]:
# hành đồng từ a-e
actions = np.array(['a', 'b', 'c', 'd', 'e'])

In [79]:
# hành đồng từ f-j
actions = np.array(['f', 'g', 'h', 'i'])

In [87]:
# hành đồng từ k-o
actions = np.array(['k', 'l', 'm', 'n', 'o'])

In [23]:
# hành đồng từ p-t
actions = np.array(['p', 'q', 'r', 's', 't'])

In [25]:
# hành đồng từ u-z
actions = np.array(['u', 'v', 'w', 'x', 'y'])

In [ ]:
actions = np.array(['z', 'j']) # chưa làm được vì cần thực hiện động tác động có thể sẽ cần làm bằng cách của video này https://www.youtube.com/watch?v=doDUihpj6ro
#hoặc 1 cách khác

In [ ]:
#Thu thập dữ liệu
# Số lượng ảnh sẽ chụp
dataset_size = 700

# Danh sách hướng dẫn thay đổi tay
messages = [
    "Di chuyển tay lên",
    "Di chuyển tay xuống",
    "Di chuyển tay sang trái",
    "Di chuyển tay sang phải",
    "Xoay vòng quanh màn hình",
    "Hướng tay lên",
    "Hướng tay xuống",
    "Hướng tay qua trái",
    "Hướng tay qua phải",
    "Giữ Nguyễn cổ tay và lòng bàn tay xoay dàn qua trái"
    "Giữ Nguyễn cổ tay và lòng bàn tay xoay dàn qua phải"
]

DATA_DIR = './far_data'
os.makedirs(DATA_DIR, exist_ok=True)

cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

frame_rate = 5  # Chụp ảnh 5 lần/giây (tương đương 200ms)
prev_time = time.time()  # Lưu thời gian frame trước

for action in actions:
    action_dir = os.path.join(DATA_DIR, str(action))
    os.makedirs(action_dir, exist_ok=True)
    
    frame = draw_text_vietnamese(frame, f"Hành động hiện tại: {action}", (50, 100), (0, 255, 0))
    
    # Hiển thị "Sẵn sàng"
    while True:
        ret, frame = cap.read()
        if not ret:
            continue
        
        frame = draw_text_vietnamese(frame, "Sẵn sàng? Nhấn 'Q' để bắt đầu!", (100, 50), (0, 255, 0))
        frame = draw_text_vietnamese(frame, f"Hành động hiện tại: {action}", (50, 100), (0, 255, 0))
        cv2.imshow('frame', frame)
        if cv2.waitKey(25) == ord('q'):
            break

    counter = 0
    start_time = time.time()
    message_index = 0
    switch_time = None  # Thời điểm bắt đầu hiển thị "Đổi tay"
    
    while counter < dataset_size:
        ret, frame = cap.read()
        if not ret:
            continue  # Nếu lỗi camera, bỏ qua frame này

        elapsed_time = time.time() - start_time

        # Đổi hướng dẫn mỗi 5 giây
        if elapsed_time > 7:
            message_index += 1
            start_time = time.time()
            
            if message_index == len(messages):
                switch_time = time.time()  # Đánh dấu thời điểm bắt đầu "Đổi tay"
                message_index = 0  

        # Nếu đang ở trạng thái "Đổi tay", hiển thị 2 giây
        if switch_time and time.time() - switch_time < 2:
            current_message = "Đổi tay"
        else:
            current_message = messages[message_index]
            switch_time = None  # Reset trạng thái

        # Hiển thị hành động & hướng dẫn
        frame = draw_text_vietnamese(frame, current_message, (50, 50), (255, 0, 0))
        
        cv2.imshow('frame', frame)

        # Chụp ảnh mỗi 200ms (5 ảnh/giây), nhưng KHÔNG chụp khi đang hiển thị "Đổi tay"
        if not switch_time or (time.time() - switch_time >= 2):
            if time.time() - prev_time > 0.2:  
                file_path = os.path.join(action_dir, f"{counter}.jpg")
                cv2.imwrite(file_path, frame)
                counter += 1
                prev_time = time.time()  # Cập nhật thời gian chụp ảnh

        cv2.waitKey(1)  # Giữ FPS cao

cap.release()
cv2.destroyAllWindows()


In [9]:
actions = np.array(['b'])

In [10]:
# Thêm dataset bị thiếu mà ko cần ghi lại toàn bộ
# Số lượng ảnh cần thêm vào
add_dataset_size = 70

DATA_DIR = './data'
os.makedirs(DATA_DIR, exist_ok=True)

cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

frame_rate = 5  # Chụp ảnh 5 lần/giây (tương đương 200ms)
prev_time = time.time()  # Lưu thời gian frame trước

for action in actions:
    action_dir = os.path.join(DATA_DIR, str(action))
    os.makedirs(action_dir, exist_ok=True)
    
    # Đếm số lượng ảnh đã có sẵn trong thư mục
    existing_images = len(os.listdir(action_dir))
    counter = existing_images  # Bắt đầu từ số ảnh hiện tại

    # Hiển thị "Sẵn sàng"
    while True:
        ret, frame = cap.read()
        if not ret:
            continue
        
        frame = draw_text_vietnamese(frame, "Sẵn sàng? Nhấn 'Q' để bắt đầu!", (100, 50), (0, 255, 0))
        cv2.imshow('frame', frame)
        if cv2.waitKey(25) == ord('q'):
            break
        
    start_time = time.time()
    message_index = 0
    switch_time = None  # Thời điểm bắt đầu hiển thị "Đổi tay"
    
    while counter < existing_images + add_dataset_size:
        ret, frame = cap.read()
        if not ret:
            continue  # Nếu lỗi camera, bỏ qua frame này

        elapsed_time = time.time() - start_time
        
        frame = draw_text_vietnamese(frame, f"Hành động hiện tại: {action}", (50, 100), (0, 255, 0))
        
        cv2.imshow('frame', frame)
        
        # Chụp ảnh mỗi 200ms (5 ảnh/giây), nhưng KHÔNG chụp khi đang hiển thị "Đổi tay"
        if not switch_time or (time.time() - switch_time >= 2):
            if time.time() - prev_time > 0.2:  
                file_path = os.path.join(action_dir, f"{counter}.jpg")
                cv2.imwrite(file_path, frame)
                counter += 1
                prev_time = time.time()  # Cập nhật thời gian chụp ảnh

        cv2.waitKey(1)  # Giữ FPS cao

cap.release()
cv2.destroyAllWindows()





# 3.Create database

In [3]:
# Khởi tạo và gắn biến xử lý bàn tay trong Mediapipe
mp_hands = mp.solutions.hands  

# khởi tạo và gắn biến vẽ hình bàn tay
mp_drawing = mp.solutions.drawing_utils  

# Khởi tạo và gắn biến hỗ trợ kiểu vẽ (có sẵn màu sắc, độ dày, v.v.)
mp_drawing_styles = mp.solutions.drawing_styles  

# Tạo một đối tượng 'Hands' để nhận diện bàn tay trong ảnh hoặc video
# static_image_mode chế độ ảnh tĩnh (True: chỉ xử lý ảnh đơn lẻ, False: xử lý video liên tục)
# min_detection_confidence Xác suất tối thiểu để mô hình chấp nhận một bàn tay
hands = mp_hands.Hands(static_image_mode=True, min_detection_confidence=0.4, max_num_hands = 1)

In [ ]:
# Định nghĩa thư mục chứa dữ liệu hình ảnh
DATA_DIR = './data'
#DATA_DIR = ["./data", "./far_data"]

# Khởi tạo danh sách để lưu trữ dữ liệu và nhãn của hình ảnh
data = []
labels = []

# Lặp qua từng thư mục con trong thư mục dữ liệu (mỗi thư mục con đại diện cho một nhãn)
for dir_ in os.listdir(DATA_DIR):
    # Lặp qua từng ảnh trong thư mục con
    for img_path in os.listdir(os.path.join(DATA_DIR, dir_)):
        data_aux = []  # Danh sách tạm để lưu tọa độ của các điểm mốc trên bàn tay

        x_ = []  # Danh sách chứa tọa độ x của các điểm mốc
        y_ = []  # Danh sách chứa tọa độ y của các điểm mốc

        # Đọc ảnh bằng OpenCV
        img = cv2.imread(os.path.join(DATA_DIR, dir_, img_path))
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Chuyển đổi sang không gian màu RGB

        # Nhận diện bàn tay trong ảnh
        results = hands.process(img_rgb)
        
        # Nếu phát hiện bàn tay trong ảnh
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                # Lưu tất cả tọa độ x, y của các điểm mốc
                for i in range(len(hand_landmarks.landmark)):
                    x = hand_landmarks.landmark[i].x
                    y = hand_landmarks.landmark[i].y

                    x_.append(x)
                    y_.append(y)

                # Chuẩn hóa dữ liệu bằng cách trừ đi giá trị nhỏ nhất của tọa độ x, y
                for i in range(len(hand_landmarks.landmark)):
                    x = hand_landmarks.landmark[i].x
                    y = hand_landmarks.landmark[i].y
                    data_aux.append(x - min(x_))  # Chuẩn hóa tọa độ x
                    data_aux.append(y - min(y_))  # Chuẩn hóa tọa độ y

            # Lưu dữ liệu đầu vào và nhãn tương ứng
            data.append(data_aux)
            labels.append(dir_)  # Nhãn là tên thư mục chứa ảnh

# Lưu dữ liệu vào file pickle để sử dụng sau này
with open('data.pickle', 'wb') as f:
    pickle.dump({'data': data, 'labels': labels}, f)

In [ ]:
# Kiểm tra xem có ảnh có khác 42 đặc trung không
for i, d in enumerate(data):
    if len(d) != 42:  # So sánh độ dài
        print(f"Ảnh {i}: {len(d)} đặc trưng")

In [ ]:
#tìm ảnh có 2 bàn tay
DATA_DIR = "./data"

# Đọc dữ liệu từ file pickle
with open('data.pickle', 'rb') as f:
    dataset = pickle.load(f)
    
data = dataset['data']  # Dữ liệu đặc trưng
labels = dataset['labels']  # Nhãn tương ứng

index = 0  # Biến đếm để tìm vị trí ảnh

# Lặp qua từng thư mục trong DATA_DIR (a-z, 0-9)
for dir_ in os.listdir(DATA_DIR):
    folder_path = os.path.join(DATA_DIR, dir_)

    if not os.path.isdir(folder_path):  # Bỏ qua nếu không phải thư mục
        continue

    # Lặp qua từng ảnh trong thư mục
    for idx, img_name in enumerate(os.listdir(folder_path)):
        img_path = os.path.join(folder_path, img_name)  # Đường dẫn ảnh

        # Kiểm tra xem ảnh có đủ 42 đặc trưng không
        if index < len(data) and len(data[index]) != 42:
            print(f"Ảnh lỗi: {img_name} | Thư mục: {dir_} | {len(data[index])} đặc trưng")

        index += 1  # Tăng biến đếm để khớp với danh sách `data`


In [143]:
# Xóa những ảnh có đặc trưng khác 42
# Load dữ liệu
with open('./data.pickle', 'rb') as f:
    data_dict = pickle.load(f)

data = data_dict['data']
labels = data_dict['labels']

# Xác định các index có số đặc trưng sai
invalid_indices = [1398, 3167, 3377, 7098, 17626, 22033]

# Lọc lại dữ liệu, bỏ qua các index sai
filtered_data = [d for i, d in enumerate(data) if i not in invalid_indices]
filtered_labels = [l for i, l in enumerate(labels) if i not in invalid_indices]

# Lưu lại file mới không chứa ảnh lỗi
new_data_dict = {'data': filtered_data, 'labels': filtered_labels}

with open('./data_cleaned.pickle', 'wb') as f:
    pickle.dump(new_data_dict, f)

print("Đã loại bỏ ảnh lỗi và lưu dữ liệu mới vào data_cleaned.pickle")


Đã loại bỏ ảnh lỗi và lưu dữ liệu mới vào data_cleaned.pickle


# 4. Train model

In [ ]:
data_dict = pickle.load(open('./data.pickle', 'rb'))

#gán mảng
data = np.asarray(data_dict['data'])
labels = np.asarray(data_dict['labels'])

# tách tệp dataset và testset
x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, shuffle=True, stratify=labels)

In [197]:
model = RandomForestClassifier()

model.fit(x_train, y_train)

y_predict = model.predict(x_test)

score = accuracy_score(y_predict, y_test)

print('{}% correctly !'.format(score * 100))

f = open('model.p', 'wb')
pickle.dump({'model': model}, f)
f.close()

93.71017471736897% correctly !


# 5. Test model

In [11]:
print(model.classes_) 

['0' '1' '2' '3' '4' '5' '6' '7' '8' '9' 'a' 'b' 'c' 'd' 'e' 'f' 'g' 'h'
 'i' 'k' 'l' 'm' 'n' 'p' 'q' 'r' 's' 't' 'u' 'v' 'w' 'x' 'y']


In [2]:
import cv2
import numpy as np
import pickle
import mediapipe as mp
import os
import string

# Load model
model_dict = pickle.load(open('./RFCmodel.p', 'rb'))
model = model_dict['model']

# Khởi tạo webcam
cap = cv2.VideoCapture(0)

# Mediapipe hands
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
hands = mp_hands.Hands(static_image_mode=True, min_detection_confidence=0.3)

# Danh sách ký tự từ 0-9 và a-z
char_list = list(string.digits + string.ascii_lowercase)
labels_dict = {char: char for char in char_list}

# Tạo thư mục lưu ảnh nếu chưa có
save_dir = "test"
os.makedirs(save_dir, exist_ok=True)

# Tìm số ảnh đã lưu để tiếp tục đánh số
existing_files = [f for f in os.listdir(save_dir) if f.endswith(".jpg")]
if existing_files:
    existing_numbers = [int(f.split(".")[0]) for f in existing_files]
    img_counter = max(existing_numbers) + 1
else:
    img_counter = 0

while True:
    data_aux = []
    x_ = []
    y_ = []

    ret, frame = cap.read()
    if not ret:
        break

    H, W, _ = frame.shape
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(frame_rgb)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(
                frame, hand_landmarks, mp_hands.HAND_CONNECTIONS,
                mp_drawing_styles.get_default_hand_landmarks_style(),
                mp_drawing_styles.get_default_hand_connections_style()
            )

        for hand_landmarks in results.multi_hand_landmarks:
            for i in range(len(hand_landmarks.landmark)):
                x = hand_landmarks.landmark[i].x
                y = hand_landmarks.landmark[i].y

                x_.append(x)
                y_.append(y)

            for i in range(len(hand_landmarks.landmark)):
                x = hand_landmarks.landmark[i].x
                y = hand_landmarks.landmark[i].y
                data_aux.append(x - min(x_))
                data_aux.append(y - min(y_))

        x1 = int(min(x_) * W) - 10
        y1 = int(min(y_) * H) - 10
        x2 = int(max(x_) * W) + 10
        y2 = int(max(y_) * H) + 10

        # Thêm padding mở rộng
        extra_padding_x = 55
        extra_padding_y = 80
        x1_ext = max(x1 - extra_padding_x, 0)
        y1_ext = max(y1 - extra_padding_y, 0)
        x2_ext = min(x2 + extra_padding_x, W)
        y2_ext = min(y2 + extra_padding_y, H)

        if len(data_aux) != 42:
            continue  # Bỏ qua vòng lặp nếu có nhiều hơn 1 bàn tay trong khung

        prediction = model.predict([np.asarray(data_aux)])
        predicted_character = labels_dict[str(prediction[0])]

        cv2.rectangle(frame, (x1 - 20, y1), (x2 + 25, y2 + 20), (0, 0, 0), 4)
                        # Hiển thị kết quả dự đoán trên màn hình
        cv2.putText(frame, f"RFC Model ", (x1, y1 - 50),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3, cv2.LINE_AA)
        cv2.putText(frame, f"Detec: {predicted_character}", (x1, y1 - 20),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3, cv2.LINE_AA)

        # Vẽ thêm khung mở rộng màu xanh dương
        cv2.rectangle(frame, (x1_ext, y1_ext), (x2_ext, y2_ext), (255, 0, 0), 2)

    cv2.putText(frame, "Press SPACE to capture, 'q' to exit", (50, 50),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

    cv2.imshow('Sign Language Detection', frame)

    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break
    elif key == ord(' '):  # Chụp ảnh khi nhấn SPACE
        if results.multi_hand_landmarks:
            hand_region = frame[y1_ext:y2_ext, x1_ext:x2_ext]
            img_path = os.path.join(save_dir, f"{img_counter}.jpg")
            cv2.imwrite(img_path, hand_region)
            print(f"Saved {img_path}")
            img_counter += 1

cap.release()
cv2.destroyAllWindows()


Saved test\105.jpg
Saved test\106.jpg
Saved test\107.jpg
Saved test\108.jpg
Saved test\109.jpg
Saved test\110.jpg
Saved test\111.jpg
Saved test\112.jpg
Saved test\113.jpg
Saved test\114.jpg
Saved test\115.jpg
Saved test\116.jpg


In [12]:
import pickle
import os
import cv2
import mediapipe as mp
import numpy as np
import string

# Load model
model_dict = pickle.load(open('./RFCmodel.p', 'rb'))
model = model_dict['model']

# Load nhãn (nếu có)
labels_dict = model_dict.get('labels', {})

# Khởi tạo MediaPipe
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=True, max_num_hands=1)

# Đường dẫn đến thư mục chứa ảnh test
test_folder = "pic_for_test"

# Danh sách ký tự từ 0-9 và a-z
char_list = list(string.digits + string.ascii_lowercase)
labels_dict = {char: char for char in char_list}

def extract_features(image):
    """Trích xuất đặc trưng keypoints bàn tay từ ảnh."""
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = hands.process(image_rgb)

    if results.multi_hand_landmarks:
        hand_landmarks = results.multi_hand_landmarks[0]
        features = []
        for lm in hand_landmarks.landmark:
            features.append(lm.x)
            features.append(lm.y)
        return np.array(features)  # Trả về đặc trưng (42 giá trị)
    else:
        return None  # Không phát hiện bàn tay

# Duyệt qua từng file trong thư mục
for file_name in os.listdir(test_folder):
    file_path = os.path.join(test_folder, file_name)

    # Đọc ảnh
    image = cv2.imread(file_path)
    if image is None:
        # print(f"Không thể đọc ảnh: {file_name}")
        continue

    # Trích xuất đặc trưng
    features = extract_features(image)
    if features is None:
        # print(f"Không tìm thấy bàn tay trong ảnh: {file_name}")
        continue

    # Dự đoán
    prediction = model.predict([features])  # Sửa lỗi: Thêm [features]
    predicted_character = labels_dict[str(prediction[0])]

    print(f"Ảnh: {file_name} -> Dự đoán: {predicted_character}")


Ảnh: a.jpg -> Dự đoán: 5
Ảnh: b.jpg -> Dự đoán: 5
Ảnh: c.jpg -> Dự đoán: 8
Ảnh: d.jpg -> Dự đoán: l
Ảnh: e.jpg -> Dự đoán: l
Ảnh: f.jpg -> Dự đoán: 8
Ảnh: g.jpg -> Dự đoán: 3
Ảnh: h.jpg -> Dự đoán: 3
Ảnh: i.jpg -> Dự đoán: 7
Ảnh: k.jpg -> Dự đoán: 3
Ảnh: l.jpg -> Dự đoán: d
Ảnh: m.jpg -> Dự đoán: l
Ảnh: n.jpg -> Dự đoán: 8
Ảnh: r.jpg -> Dự đoán: 3
Ảnh: s.jpg -> Dự đoán: l
Ảnh: t.jpg -> Dự đoán: 3
Ảnh: u.jpg -> Dự đoán: 3
Ảnh: v.jpg -> Dự đoán: 3
Ảnh: w.jpg -> Dự đoán: 7
Ảnh: x.jpg -> Dự đoán: l
Ảnh: y.jpg -> Dự đoán: 7


# Note có 1 số ký tự giống như O và 0 , W và 6 vì vậy phải xem xét xem là đang nói về đề tài nào để phân biệt 
# Sau khi hoàn thành nghiên cứu natural language để cải thiện đầu ra và tự nhận biết được nhưng ký tự tương đồng